In [4]:
from utils.load_results import *
from utils.plot_helpers import *

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import torch
from utils.analysis_from_interaction import *
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim
from egg.core.callbacks import Checkpoint, CheckpointSaver

In [15]:
checkpoint = torch.load('results/(3,16)_game_size_10_vsf_3/standard/0/final.tar', map_location=torch.device('cpu'))

In [22]:
checkpoint.model_state_dict.keys()

odict_keys(['sender.sos_embedding', 'sender.agent.fc1.weight', 'sender.agent.fc1.bias', 'sender.agent.fc2.weight', 'sender.agent.fc2.bias', 'sender.agent.fc3.weight', 'sender.agent.fc3.bias', 'sender.hidden_to_output.weight', 'sender.hidden_to_output.bias', 'sender.embedding.weight', 'sender.embedding.bias', 'sender.cell.weight_ih', 'sender.cell.weight_hh', 'sender.cell.bias_ih', 'sender.cell.bias_hh', 'receiver.agent.fc1.weight', 'receiver.agent.fc1.bias', 'receiver.cell.weight_ih', 'receiver.cell.weight_hh', 'receiver.cell.bias_ih', 'receiver.cell.bias_hh', 'receiver.embedding.weight', 'receiver.embedding.bias'])

In [23]:
checkpoint.model_state_dict['sender.agent.fc1.weight']

tensor([[-0.9174,  1.4376,  0.1796,  ...,  0.6386,  0.4370,  0.0163],
        [ 0.0842, -0.0322,  0.4671,  ..., -0.4641,  0.3651,  0.1083],
        [ 0.6989, -0.4786,  1.3227,  ...,  0.2486,  0.6486, -0.2115],
        ...,
        [ 0.1716, -0.1162, -0.1908,  ...,  0.2515,  0.1634,  0.1638],
        [-0.0442,  0.8096, -0.6185,  ...,  0.1850, -0.2574, -0.0300],
        [-0.6278,  0.7457,  0.1487,  ..., -0.7957, -0.1645,  0.5329]])